In [ ]:
import requests
import time
from datetime import datetime

In [ ]:
%load_ext jupyternotify

In [ ]:
pip install twilio

In [ ]:
import os
from twilio.rest import Client

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = 'SID'
auth_token = 'AUTH_TOKEN'
client = Client(account_sid, auth_token)

In [ ]:
state = 'Rajasthan'
district = 'Ajmer'
phone_number = '+91XXXXXXXXXX'
twilio_trial_number = '+1XXXXXXXXXX'

In [ ]:
headers = {'Content-Type': 'application/json',
               'origin': 'https://www.cowin.gov.in',
              'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

In [ ]:
states = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states", headers=headers).json()
state_id = [a for a in states['states'] if a['state_name'] == state][0]['state_id']
districts = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id), headers=headers).json()
district_id = [a for a in districts['districts'] if a['district_name'] == district][0]['district_id']

In [ ]:
# %%notify -m "Cell Complete"
while True:
    api = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(district_id, datetime.today().strftime('%d-%m-%Y'))
    should_break = False
    abc = requests.get(api, headers=headers)
    should_continue = True
    print(str(abc.status_code) + ' ' + datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    if abc.ok:
        center_ob = abc.json()
        for center in center_ob['centers']:
            for session in center['sessions']:
                if session['min_age_limit'] == 18:
                    print(str(session['vaccine'])+ '       ' + str(session['available_capacity']) + ' '+ session['date'])
                    if session['available_capacity'] > 0:
                        %notify -m "Slot is Open"
                        message = client.messages \
                            .create(
                                 body='Slot is Open for [{}] Capacity [{}] Time: [{}]'.format(session['vaccine'], session['available_capacity'], datetime.now().strftime("%m/%d/%Y, %H:%M:%S")),
                                 from_=twilio_trial_number,
                                 to=phone_number
                             )
                        print(message.sid)
                        should_break = True
        if should_break:
            break
        time.sleep(5)
    else:
        for i in range(10):
            abc = requests.get(api, headers=headers)
            print(abc)
            if abc.ok:
                should_continue = True
                break
            else:
                should_continue = False
            time.sleep(5)
        if not should_continue:
            %notify -m "Response is 401"
            break

In [ ]:
'Slot is Open for {} Time: {}'.format('blabla', datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

In [ ]:
pip install twilio